## Notebook 04: Sales Agent
markdown
markdown
## Model Integration, Narrative Resonance & Fairness (prototype)

- Goal: show a small, auditable prototype that approximates narrative resonance and urgency without requiring an LLM.
- Approach: use a lightweight linear surrogate so feature attributions are exact (weight * feature), which serves as a SHAP-like explanation for this prototype.
- Fairness rule: apply a penalty to urgency when credit score is below a threshold; the penalty scales with the urgency contribution so the effect is interpretable.
- Note: this cell does not load any LoRA or Llama model. It demonstrates the integration points and the explanation+penalty pipeline in a reproducible way.
code
python
# Prototype linear 'NarrativeModel' with exact feature contributions
from dataclasses import dataclass
def text_features(text):
    t = text.lower()
    return {
        'story_future': 1.0 if 'future' in t or 'plan' in t or 'start' in t or 'next' in t else 0.0,
        'urgency': 1.0 if 'urgent' in t or 'soon' in t or 'priority' in t else 0.0,
        'emotional_valence': 1.0 if 'excited' in t or 'love' in t or 'great' in t else 0.0,
    }
@dataclass
class NarrativeModel:
    # weights for a linear surrogate; resonance and urgency share features but have separate weights
    w_resonance: dict = None,
    def __post_init__(self):
        if self.w_resonance is None:
            self.w_resonance = {'story_future': 0.6, 'urgency': 0.1, 'emotional_valence': 0.3}
        if self.w_urgency is None:
            self.w_urgency = {'story_future': 0.0, 'urgency': 0.8, 'emotional_valence': 0.2}
    def predict(self, text):
        f = text_features(text)
        contrib_res = {k: self.w_resonance[k]*f[k] for k in f}
        contrib_urg = {k: self.w_urgency[k]*f[k] for k in f}
        resonance = sum(contrib_res.values())
        urgency = sum(contrib_urg.values())
        return {'resonance': resonance, 'urgency': urgency, 'contrib_res': contrib_res, 'contrib_urg': contrib_urg}
# small demo
model = NarrativeModel()
examples = [ 'I plan to start next month and am excited about the future', 'This is urgent, we need priority support', 'Curious about pricing, not urgent' ]
for ex in examples:
    out = model.predict(ex)
markdown
#VSC-45de753f
markdown
## Notebook 04 — Sales Agent

Brief: a compact, auditable reference implementation of a sales assistant pipeline. It demonstrates deterministic scoring, persona templates, finite-state conversation flows, offer generation, analytics, and a small, transparent prototype for narrative resonance and fairness (surrogate model).

- **Purpose:** surface qualified opportunities reliably, audibly, and reproducibly.
- **Design principles:** deterministic transforms; interpretability (explicit arithmetic); auditability (append-only events); reproducibility (same inputs → same outputs).
- **Prototype note:** the narrative resonance model is a linear surrogate with exact per-feature attributions (weight × feature). This enables a SHAP-like explanation and a simple fairness penalty that reduces urgency when credit < threshold.

Outputs: runnable examples, visualizations, and JSON artifacts written to the `artifacts/` folder.
code
#VSC-3e7e7b85
python
# Persona example: simple template mapping
markdown
## Notebook 04 — Sales Agent

Brief: a compact, auditable reference implementation of a sales assistant pipeline. It demonstrates deterministic scoring, persona templates, finite-state conversation flows, offer generation, analytics, and a small, transparent prototype for narrative resonance and fairness (surrogate model).

- **Purpose:** surface qualified opportunities reliably, audibly, and reproducibly.
- **Design principles:** deterministic transforms; interpretability (explicit arithmetic); auditability (append-only events); reproducibility (same inputs → same outputs).
- **Prototype note:** the narrative resonance model is a linear surrogate with exact per-feature attributions (weight × feature). This enables a SHAP-like explanation and a simple fairness penalty that reduces urgency when credit &lt; threshold.

Outputs: runnable examples, visualizations, and JSON artifacts written to the `artifacts/` folder.
code
# Setup: imports and artifacts directory
import json, math, random
from pathlib import Path
import matplotlib.pyplot as plt
import networkx as nx
ARTIFACTS = Path('artifacts')
ARTIFACTS.mkdir(parents=True, exist_ok=True)
print('Environment ready. Artifacts dir:', ARTIFACTS)
print('matplotlib backend:', plt.get_backend())

markdown
## Persona — Tone, Claims, Fallbacks

Purpose: keep responses consistent, safe, and testable.

- **Styles:** `informative`, `consultative`, `escalate`.
- **Mapping:** deterministic feature → score → argmax selects style.
- **Templates:** small, reviewable strings; escalate when confidence is low.

Example: the persona is a tiny policy object (enum + templates) that is exhaustively testable.
code
# Persona example: simple template mapping
from dataclasses import dataclass, asdict
@dataclass
class Persona:
    name: str
    tone: str
    templates: dict
p = Persona(name='SalesPro', tone='consultative', templates={'greet': 'Hi {name}, thanks for reaching out.', 'followup': 'Are you available for a short call this week?'})
print('Persona preview:')
print(asdict(p))
# render a template
print('

markdown
## Core Conversation Flows

Purpose: control dialog progression with small, verifiable state machines.

- **Representation:** directed graph where nodes are states and edges have predicate guards.
- **Guarantees:** analyze reachability and liveness; choose transitions by threshold or argmax when using scores.
- **Benefit:** explicit flows prevent unexpected statements and simplify testing.

Below: a minimal flow visualization and runnable FSM example.
code
# Simple FSM flow and visualization
G = nx.DiGraph()
states = ['start','qualify','proposal','handoff','closed']
G.add_nodes_from(states)
G.add_edge('start','qualify')
G.add_edge('qualify','proposal')
G.add_edge('proposal','closed')
G.add_edge('proposal','handoff')
pos = nx.spring_layout(G, seed=2)
plt.figure(figsize=(6,4))
nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=1200, arrowsize=20)
plt.title('Conversation Flow (simple)')
plt.show()
print('Flow nodes:', list(G.nodes()))

markdown
## Lead Qualification — Deterministic Scoring

Purpose: convert text + metadata into a compact numeric score for gating and prioritization.

- **Features:** budget, timeline, decision-maker signals.
- **Model:** interpretable linear scoring:

$$score = w_1 x_1 + w_2 x_2 + \dots + w_n x_n$$

- Notes: weights are small and explainable; apply a sigmoid to convert to probability if needed.

Below: a deterministic scorer, example outputs, and a simple visualization.
code
# Simple deterministic lead scorer
def extract_features(text):
    t = text.lower()
    return {
        'budget': 1 if 'budget' in t or '$' in t else 0,
        'timeline': 1 if 'soon' in t or 'next' in t or 'month' in t else 0,
        'decision_maker': 1 if 'i am' in t or 'we are' in t else 0
    }
def score_lead(text, weights=(0.5,0.3,0.2)):
    f = extract_features(text)
    vals = [f['budget'], f['timeline'], f['decision_maker']]
    s = sum(w*v for w,v in zip(weights, vals))
    return s, f
examples = [ 'We have a $5000 budget and want to start next month', 'Curious about pricing', 'I am the manager and want a demo soon' ]
scores = [score_lead(t) for t in examples]
print('Lead scores and features:')
for t,(s,f) in zip(examples,scores):
    print('
    print('SCORE:', s, 'FEATURES:', f)
# plot bar chart
plt.figure(figsize=(6,3))
plt.bar(range(len(scores)), [s for s,_ in scores], tick_label=[ 'ex1','ex2','ex3' ])
plt.title('Lead Scores')
plt.ylim(0,1.1)
plt.show()

markdown
## Objection Handling

Purpose: map common objections to short, safe rebuttals.

- **Taxonomy:** class label → canonical response template.
- **Escalation:** offer human handoff when confidence is low or request is out-of-scope.
- **Safety:** keep responses concise and verifiable.

Below: a small mapping and examples.
code
# Objection mapping demo
objections = {
    'price': 'I understand pricing is important. Can I show value relevant to your goals?',
    'timing': 'I hear timing is tight. Would a short trial or phased roll-out help?',
    'needs': 'Thanks for sharing that. Can you tell me which features matter most?'
}
sample = ['price','needs','other']
for s in sample:
    resp = objections.get(s, 'I will connect you with a specialist for that request.')
    print(f'Objection: {s} -> Response: {resp}')

markdown
## Offer Generation and Pricing

Purpose: produce deterministic, auditable quotes from structured inputs.

- **Inputs:** quantity `q`, unit price `p`, discount `d`.
- **Formula:** $$total = q \cdot p \cdot (1 - d)$$
- **Design:** expose every intermediate value (subtotal, discount) for auditing.

Below: example quote computation and breakdown visualization.
code
# Offer generation demo
def quote(q, p, d=0.0):
    subtotal = q*p
    discount = subtotal * d
    total = subtotal - discount
    return {'qty':q,'unit':p,'subtotal':subtotal,'discount':discount,'total':total}
q = 10; p = 199.0; d = 0.1
qout = quote(q,p,d)
print('Quote preview:', qout)
# visualize breakdown
plt.figure(figsize=(5,3))
plt.bar(['subtotal','discount','total'], [qout['subtotal'], qout['discount'], qout['total']], color=['#4c78a8','#f58518','#54a24b'])
plt.title('Quote Breakdown')
plt.show()

markdown
## Analytics, Signals, and Metrics

Purpose: monitor agent health and conversion performance.

- **Key metrics:** conversion rate, time-to-qualify, abandonment rate.
- **Instrumentation:** record events (message, qualified, handoff) in append-only timelines (see Notebook 03).
- **Use:** deterministic replays to compute metrics and compare rule changes.

Below: a small event simulation and summary chart.
code
# Simulate events and compute simple metrics
events = [ {'lead_id':1,'event':'message'},{'lead_id':1,'event':'qualified'},{'lead_id':1,'event':'closed'}, {'lead_id':2,'event':'message'},{'lead_id':2,'event':'abandoned'} ]
total = len({e['lead_id'] for e in events})
closed = len([e for e in events if e['event']=='closed'])
abandoned = len([e for e in events if e['event']=='abandoned'])
conv = closed/total
print(f'Total leads: {total}, closed: {closed}, abandoned: {abandoned}, conversion rate: {conv:.2f}')
plt.figure(figsize=(4,2))
plt.bar(['closed','abandoned'], [closed,abandoned], color=['#2ca02c','#d62728'])
plt.title('Outcomes')
plt.show()

print('\nPreview lead:')
## Integrations and Runtime

Purpose: connect internal events to external systems safely and reliably.
markdown
- **Adapters:** convert internal artifacts to CRM, calendar, or handoff messages.

































































































































543.10pythonpython3pythonPython 3Checklist before deploy: observability, alerting, error budgets, and privacy/compliance review.- Harden integrations: idempotency, retries, observability, and alerting.- Build A/B evaluation harness and offline fairness audits.- Add human-in-the-loop review and approval flows.- Replace surrogate with calibrated ML models (if GPU/inference available).Recommended follow-ups:## Next Steps & Roadmapmarkdownprint('Wrote audit artifact ->', ARTIFACTS / 'narrative_audit.json')pprint.pprint(audit)import pprint    json.dump(audit,f,indent=2)with open(ARTIFACTS / 'narrative_audit.json','w',encoding='utf-8') as f:# write audit artifact and preview    audit.append(rec)    rec = {'text': text, 'credit': credit, 'resonance': round(out['resonance'],3), 'raw_urgency': round(out['urgency'],3), 'penalized_urgency': round(v['penalized_urgency'],3), 'reason': v['reason']}    v = apply_fairness_penalty(out, credit)    out = model.predict(text)for text, credit in cases:audit = []cases = [(examples[0], 700), (examples[1], 580), (examples[2], 620)]]    'Curious about pricing, not urgent'    'This is urgent, we need priority support',    'I plan to start next month and am excited about the future',examples = [model = NarrativeModel()# Demo and compact audit    return {'raw_urgency': raw_urgency, 'penalized_urgency': penalized, 'reason': f'penalized (credit {credit_score} < {c0})', 'contrib_urgency': contrib_urgency}    penalized = max(0.0, raw_urgency - lamb * contrib_urgency)        return {'raw_urgency': raw_urgency, 'penalized_urgency': raw_urgency, 'reason':'no penalty', 'contrib_urgency': contrib_urgency}    if credit_score >= c0:    contrib_urgency = sum(out['contrib_urg'].values())    raw_urgency = out['urgency']def apply_fairness_penalty(out, credit_score, c0=600, lamb=0.8):        return {'resonance': resonance, 'urgency': urgency, 'contrib_res': contrib_res, 'contrib_urg': contrib_urg}        urgency = sum(contrib_urg.values())        resonance = sum(contrib_res.values())        contrib_urg = {k: self.w_urgency.get(k,0.0)*f.get(k,0.0) for k in f}        contrib_res = {k: self.w_resonance.get(k,0.0)*f.get(k,0.0) for k in f}        f = text_features(text)    def predict(self, text):            self.w_urgency = {'story_future': 0.0, 'urgency': 0.8, 'emotional_valence': 0.2}        if self.w_urgency is None:            self.w_resonance = {'story_future': 0.6, 'urgency': 0.1, 'emotional_valence': 0.3}        if self.w_resonance is None:    def __post_init__(self):    w_urgency: dict = None    w_resonance: dict = Noneclass NarrativeModel:@dataclass    }        'emotional_valence': 1.0 if any(w in t for w in ('excited','love','great')) else 0.0,        'urgency': 1.0 if any(w in t for w in ('urgent','soon','priority')) else 0.0,        'story_future': 1.0 if any(w in t for w in ('future','plan','start','next')) else 0.0,    return {    t = text.lower()def text_features(text):from dataclasses import dataclass# Prototype linear 'NarrativeModel' with exact feature contributionscodeNotes: this is a CPU-friendly prototype suitable for unit tests and audits. If you have model access and GPU, we can replace the surrogate with a LoRA adapter on an LLM; ask and I will add the exact `transformers` + `peft` cells and a `requirements.txt`.$$\text{penalized\_urgency} = \max\bigl(0, u - \lambda \cdot I[credit < c_0] \cdot contrib_{urgency}\bigr)$$Formula (penalized urgency):- **Fairness rule:** if `credit < c0` apply a penalty to urgency proportional to the summed urgency contribution. This is auditable and tunable.- **Surrogate model:** linear weights; per-feature contribution is exact (weight × feature) so explanations are unambiguous.Purpose: provide a small, fully transparent surrogate that approximates two behaviors: `resonance` (how well a message fits the customer's narrative) and raw `urgency`.## Narrative Resonance (Prototype) & Fairnessmarkdownprint(json.dumps(lead,indent=2))print('
Preview lead:')print('Wrote artifacts:', ARTIFACTS / 'lead_sample.json', ARTIFACTS / 'offer_sample.json')with open(ARTIFACTS / 'offer_sample.json','w',encoding='utf-8') as f: json.dump(offer,f,indent=2)with open(ARTIFACTS / 'lead_sample.json','w',encoding='utf-8') as f: json.dump(lead,f,indent=2)offer = {'id':1,'lead_id':1,'total':1791.0}lead = {'id':1,'name':'ACME','score':0.82}codeBelow: a sample lead and offer writer and a preview of the files written.- **Format:** make artifacts minimal, typed, and easy to validate.- **Store:** leads, offers, and audit records under `artifacts/`.Purpose: produce JSON artifacts that serve as contracts between components.## Artifacts & Schema Exportsmarkdownprint('Deterministic tests passed')assert score_lead('Just browsing')[0] < 0.5assert score_lead('We have a $1000 budget and can start next month')[0] > 0.5# Deterministic testscodeBelow: a few assertions that should pass deterministically.- **Value:** prevent regressions and document expected behavior.- **Tests:** small, fast, pure assertions with fixed inputs.Purpose: codify invariants and expectations for reviewers and CI.## Deterministic Tests & Worked Examplesmarkdownprint(json.dumps(payload, indent=2))print('CRM payload (preview):')payload = build_crm_payload(42,'ACME Corp',0.78)    return {'lead_id':lead_id,'name':name,'score':score}def build_crm_payload(lead_id, name, score):# Integration stub: CRM payload buildercodeBelow: a stub adapter that prints the CRM payload instead of sending it.- **Pattern:** keep core logic pure and testable; adapters should be thin I/O layers.- **Best practices:** idempotency, clear error semantics, and retries with backoff.
Suggestions for follow-up work include adding calibrated ML models for scoring, human-in-the-loop workflows, richer A/B evaluation harnesses, privacy and compliance reviews, and production hardening of integrations. Create a checklist for observability, alerting, and error budgeting before deploying.